In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-12-01 20:43:09.865320: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-01 20:43:09.867586: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-12-01 20:43:09.867594: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#preprocessing for efficient net

IMG_SIZE = 224
BATCH_SIZE = 32

# Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

# No augmentation for validation and test sets
val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Paths to train, val, and test directories
train_dir = "chest_xray/train"
val_dir = "chest_xray/validation"
test_dir = "chest_xray/test"

# Load each set
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 4694 images belonging to 2 classes.
Found 585 images belonging to 2 classes.
Found 586 images belonging to 2 classes.


In [3]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Get a batch of images and labels
# image_batch, label_batch = next(train_generator)

# # Number of images to display
# num_images = 10

# # Iterate over the batch and plot each image

# fig, axes = plt.subplots(2, 5, figsize=(15, 6))
# for i in range(num_images):
#     row = i // 5
#     col = i % 5
#     axes[row, col].imshow(image_batch[i])
#     axes[row, col].axis('off')
    
#     # Display label (0 or 1), where 0 might indicate 'Normal' and 1 'Pneumonia'
#     label = label_batch[i]
#     class_name = 'Normal' if label == 0 else 'Pneumonia'
#     axes[row, col].set_title(class_name, color='red')

# plt.tight_layout()
# plt.show()

In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Define the model
def create_resnet_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=1):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Freeze the base model layers (so they won't be trained)
    base_model.trainable = False
    
    # Add custom layers on top
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='sigmoid')  # Use sigmoid for binary classification
    ])
    
    # Compile the model
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Create the model
model = create_resnet_model()

# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with the data generators
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate on the test set
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test accuracy: {test_acc}")

2024-12-01 20:45:32.238834: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-01 20:45:32.238952: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-12-01 20:45:32.238974: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-12-01 20:45:32.238992: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-12-01 20:45:32.239010: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Epoch 1/50
147/147 [==============================] - 72s 484ms/step - loss: 0.6149 - accuracy: 0.7045 - val_loss: 0.5432 - val_accuracy: 0.7299
Epoch 2/50
147/147 [==============================] - 73s 494ms/step - loss: 0.5400 - accuracy: 0.7331 - val_loss: 0.5163 - val_accuracy: 0.7248
Epoch 3/50
147/147 [==============================] - 73s 497ms/step - loss: 0.5106 - accuracy: 0.7458 - val_loss: 0.5115 - val_accuracy: 0.7726
Epoch 4/50
147/147 [==============================] - 73s 499ms/step - loss: 0.4893 - accuracy: 0.7559 - val_loss: 0.4726 - val_accuracy: 0.7915
Epoch 5/50
147/147 [==============================] - 73s 497ms/step - loss: 0.4877 - accuracy: 0.7514 - val_loss: 0.5650 - val_accuracy: 0.7709
Epoch 6/50
147/147 [==============================] - 73s 499ms/step - loss: 0.4844 - accuracy: 0.7552 - val_loss: 0.4745 - val_accuracy: 0.7880
Epoch 7/50
147/147 [==============================] - 74s 499ms/step - loss: 0.4724 - accuracy: 0.7563 - val_loss: 0.4471 - val_ac